#  Convert a ESRI Shapefile into a JSON file and transform its geospatial coordinates

####  Import libraries

In [ ]:
import pandas as pd
import numpy as np

from json import dumps

import shapefile #provides support to handle ESRI Shapefiles in pure Python.

import utm #allows to convert geospatial coordinates from UTM to GPS format

import unidecode #allows to remove accents in strings

The ESRI shapefile that is going to be transformed contains data from the negihbourhoods of Barcelona . It can be downloaded from https://w20.bcn.cat/cartobcn/default.aspx?lang=en by creating a free account<br>
<br>
ESRI format contains a shapefile (shp) which contains the shapes in vectorial format of the different neighbourhoods and districs of the city, and a database file (dfx) which contain some statistical data of every neighbourhood. They can be easily read with the Python library pyshp (https://pypi.org/project/pyshp/) <br>
<br>
The database contains the following fields: <br>
o District: District code  <br>
o NDistric: District name  <br>
o CBarri: Neighbourhood code  <br>
o NBarri: neighbourhood name  <br>
o Homes: Number of men  <br>
o Dones: Number of women. <br>
o Perimetr: Perimiter <br>
o Area: Surface in square meters  <br>
o Coord_X: X-axis centroid coordinates, in UTM format<br>
o Coord_Y: Y-axis centroid coordinates, in UTM format<br>
o Web1: Link to district's web page  <br>
o Web2: Link to city's annual stats  <br>
o Web3: Link to districts's annual stats  <br>
o Web4: Link to neoighboourhood's annual stats  <br>
<br>
A shapefile file with vectorial information of neighbourhood boundaries will be used to create city maps with Cloropeth library and Coord_X and Coord_Y files, as they define of the centroid of each neighbourhood, will be used to make queries with Foursquare API. However downloaded data use UTM format, and we need to convert it to GPS coordenates. Therefore we are going to transform the files.

Define define input and ouput filename parameters

In [2]:
raw_neighbourhood_file = 'raw_data/BCN_Barri_ED50_SHP'
processed_neighbourhood_file = 'processed_data/BCN_Barri_ED50_SHP'    
processed_neighbourhood_jsonfile = 'processed_data/BCN_Barri_ED50_SHP.json'    

In [3]:
#Using PyShp create a Reader object to access the data from the Barcelona Shapefile                
shp_r = shapefile.Reader(raw_neighbourhood_file)

#Create a Writer object to write data to as a new Shapefile
shp_w = shapefile.Writer(processed_neighbourhood_file)   

#Set variables for access to the field information of both the original and new Shapefile. We visualize the name of the fields in Catalan that dataset contains
shp_fields_r = shp_r.fields
shp_fields_w = shp_w.fields

for name in shp_fields_r:
    if type(name) == "tuple":
        continue
    elif name[0] == "C_Distri":
        args = name
        args[0] = 'C_Dist'
        shp_w.field(*args)
    elif name[0] == "N_Distri":
        args = name
        args[0] = 'N_Dist'
        shp_w.field(*args)
    elif name[0] == "C_Barri":
        args = name
        args[0] = 'C_Neigh'
        shp_w.field(*args)
    elif name[0] == "N_Barri":
        args = name
        args[0] = 'N_Neigh'
        shp_w.field(*args)
    elif name[0] == "Homes":
        args = name
        args[0] = 'Men' 
        shp_w.field(*args)
    elif name[0] == "Dones":
        args = name
        args[0] = 'Women' 
        shp_w.field(*args)
    elif name[0] == "Perim":
        args = name
        args[0] = 'Perimeter' 
        shp_w.field(*args)
    elif name[0] == "Coord_X":
        args = ['Longitude', 'F', 8, 16]
        shp_w.field(*args)
    elif name[0] == "Coord_Y":
        args = ['Latitude', 'F', 8, 16]     
        shp_w.field(*args)
    else:
        args = name
        shp_w.field(*args)

In [4]:
shp_r.encoding = 'Latin-1' #Encoding for Western languages such as Catalan
records = shp_r.records()

for row in records:
    args = row
    args[1] = str.title(unidecode.unidecode(args[1]))
    args[3] = str.title(unidecode.unidecode(args[3]))
    args[9],args[8] = utm.to_latlon(args[8], args[9], 31, 'T')
    shp_w.record(*args)

In [5]:
for feature in shp_r.shapes():
    # if there is only one part
    if len(feature.parts) == 1:
        # create empty list to store all the coordinates
        poly_list = []
        # get each coord that makes up the polygon
        for coords in feature.points:
            x, y = coords[0], coords[1]
            # tranform the coord
            #new_x, new_y = transform(input_projection, output_projection, x, y)
            new_y, new_x = utm.to_latlon(x, y, 31, 'T')
            # put the coord into a list structure
            poly_coord = (float(new_x), float(new_y))
            # append the coords to the polygon list
            poly_list.append(poly_coord)     
        # add the geometry to the shapefile.
        shp_w.poly([poly_list])
    else:
        # append the total amount of points to the end of the parts list
        feature.parts.append(len(feature.points))
        # enpty list to store all the parts that make up the complete feature
        poly_list = []
        # keep track of the part being added
        parts_counter = 0

        # while the parts_counter is less than the amount of parts
        while parts_counter < len(feature.parts) - 1:
            # keep track of the amount of points added to the feature
            coord_count = feature.parts[parts_counter]
            # number of points in each part
            no_of_points = abs(feature.parts[parts_counter] - feature.parts[parts_counter + 1])
            # create list to hold individual parts - these get added to poly_list[]
            part_list = []
            # cut off point for each part
            end_point = coord_count + no_of_points

            # loop through each part
            while coord_count < end_point:
                for coords in feature.points[coord_count:end_point]:
                    x, y = coords[0], coords[1]
                    # tranform the coord
                    #new_x, new_y = transform(input_projection, output_projection, x, y)
                    new_y, new_x = utm.to_latlon(x, y, 31, 'T')
                    # put the coord into a list structure
                    poly_coord = [float(new_x), float(new_y)]
                    # append the coords to the part list
                    part_list.append(poly_coord)
                    coord_count = coord_count + 1
            # append the part to the poly_list
            poly_list.append(part_list)
            parts_counter = parts_counter + 1
        # add the geometry to to new file
        shp_w.poly(poly_list)


In [6]:
shp_w.close()

In [7]:
#read file, parse out the records and shapes
shp_r = shapefile.Reader(processed_neighbourhood_file)
shp_r.encoding = 'Latin-1' #Encoding for Western languages such as Catalan

#grab the shapefile's field names (omit the first psuedo field)
fields = [x[0] for x in shp_r.fields][1:]
#records = sf.records()
records = [r[:] for r in shp_r.records()]
shps = [s.points for s in shp_r.shapes()]

#write the records into a dataframe
bcn_neigh_df = pd.DataFrame(columns=fields, data=records)

#add the coordinate data to a column called "coords"
bcn_neigh_df = bcn_neigh_df.assign(coords=shps)

#Visualize the first rows of the file
bcn_neigh_df.head()

,C_Dist,N_Dist,C_Neigh,N_Neigh,Men,Women,Area,Perimeter,Longitude,Latitude,WEB_1,WEB_2,WEB_3,WEB_4,coords
0,01,Ciutat Vella,01,El Raval,26553,21850,1.098393e+06,5557.372878,2.171593,41.38081,http://www.bcn.cat/ciutatvella,http://www.bcn.cat/estadistica/catala/dades/in...,http://www.bcn.cat/estadistica/catala/dades/gu...,http://www.bcn.cat/estadistica/catala/dades/in...,"[(2.17114583251087, 41.387318300967955), (2.17..."
1,01,Ciutat Vella,02,El Barri Gotic,8368,7508,8.419051e+05,5683.004856,2.178549,41.38294,http://www.bcn.cat/ciutatvella,http://www.bcn.cat/estadistica/catala/dades/in...,http://www.bcn.cat/estadistica/catala/dades/gu...,http://www.bcn.cat/estadistica/catala/dades/in...,"[(2.183565559937503, 41.38311899420626), (2.18..."
2,01,Ciutat Vella,03,La Barceloneta,7581,7631,1.313868e+06,13039.266793,2.191261,41.37905,http://www.bcn.cat/ciutatvella,http://www.bcn.cat/estadistica/catala/dades/in...,http://www.bcn.cat/estadistica/catala/dades/gu...,http://www.bcn.cat/estadistica/catala/dades/in...,"[(2.20082586629123, 41.38678345631065), (2.200..."
3,01,Ciutat Vella,04,"Sant Pere, Santa Caterina I La Ribera",11466,11390,1.114299e+06,4658.031512,2.184539,41.38864,http://www.bcn.cat/ciutatvella,http://www.bcn.cat/estadistica/catala/dades/in...,http://www.bcn.cat/estadistica/catala/dades/gu...,http://www.bcn.cat/estadistica/catala/dades/in...,"[(2.1834921087968935, 41.39327449798472), (2.1..."
4,02,Eixample,05,El Fort Pienc,15039,16924,9.289010e+05,4175.971325,2.182589,41.39926,http://www.bcn.cat/eixample,http://www.bcn.cat/estadistica/catala/dades/in...,http://www.bcn.cat/estadistica/catala/dades/gu...,http://www.bcn.cat/estadistica/catala/dades/in...,"[(2.1834921087968935, 41.39327449798472), (2.1..."


We create a processed GeoJSON file which it is going to be used to create map plots with Chloropeth library later

In [8]:
processed_neighbourhood_jsonfile = 'processed_data/BCN_Barri_ED50_SHP.json'

# read the shapefile
reader = shapefile.Reader(processed_neighbourhood_file)
fields = reader.fields[1:]
field_names = [field[0] for field in fields]
buffer = []
for sr in reader.shapeRecords():
    atr = dict(zip(field_names, sr.record))
    geom = sr.shape.__geo_interface__
    buffer.append(dict(type="Feature", \
    properties=atr, geometry=geom)) 

# write the GeoJSON file
geojson = open(processed_neighbourhood_jsonfile, "w")
geojson.write(dumps({"type": "FeatureCollection",\
"features": buffer}, indent=2) + "\n")
geojson.close()